In [ ]:
import sys
sys.path.append("../")
import uproot
import hist
import numpy as np
import mplhep as hep
from matplotlib import pyplot as plt
from cycler import cycler
from coffea import util
from azh_analysis.utils.parameters import get_lumis

hep.style.use(["CMS", "fira", "firamath"])
cats = ["em", "et", "mt", "tt"]
year, mass = "2018", "300"
lumi = get_lumis()[year]

In [ ]:
mc = uproot.open(f"/uscms_data/d3/jdezoort/AZh_columnar/CMSSW_10_2_9/src/azh_coffea/src/notebooks/root_for_combine/MC_0btag_2018.root")
keys = [k.strip(";1") for k in mc.keys()]
keys

In [ ]:
signal = util.load(f'../output/signal_UL_2018_all_OS.coffea')
h = signal["m4l"]["GluGluToAToZhToLLTauTauM225"][::sum, "eemt", 0, "nom", "cons", :]
print(h.plot1d())
print(h.variances())

In [ ]:
mass_points = [
    '225','250','275','300','325','350','375','400','450',
    '500','600','700','750','800','900','1000',
    '1200','1400','1600','1800','2000'
]
for m in mass_points:
    signal_0 = uproot.open(f"root_for_combine/signal_{m}_0btag_{year}.root")
    signal_1 = uproot.open(f"root_for_combine/signal_{m}_btag_{year}.root")
    for cat in cats:
        signal_0_ggA = signal_0["ee"+cat+"/ggA;1"].to_hist() + signal_0["mm"+cat+"/ggA;1"].to_hist()
        signal_0_bbA = signal_0["ee"+cat+"/bbA;1"].to_hist() + signal_0["mm"+cat+"/bbA;1"].to_hist()
        signal_1_ggA = signal_1["ee"+cat+"/ggA;1"].to_hist() + signal_1["mm"+cat+"/ggA;1"].to_hist()
        signal_1_bbA = signal_1["ee"+cat+"/bbA;1"].to_hist() + signal_1["mm"+cat+"/bbA;1"].to_hist()
        fig, ax = plt.subplots()
        signal_0_ggA.plot1d(ax=ax, label="ggA, 0btag")
        signal_1_ggA.plot1d(ax=ax, label="ggA, btag")
        signal_0_bbA.plot1d(ax=ax, label="bbA, 0btag")
        signal_1_bbA.plot1d(ax=ax, label="bbA, btag")
        plt.show()


In [ ]:
stacks = {}
for btag in ["0btag", "btag"]:
    mc = uproot.open(f"root_for_combine/MC_{btag}_{year}.root")
    data = uproot.open(f"root_for_combine/data_{btag}_{year}.root")
    groups = [       
        "ggZZ",  "ZZ", "TTZ", "VVV", "WZ",
        "ZHtt", "TTHtt", "WHWW", "ZHWW", "ggZHWW", "ggHZZ"
    ]
    
    mc_stack = {"em": {}, "et": {}, "mt": {}, "tt": {}}
    reducible_stack = {}
    for group in groups:
        for cat in cats:
            ee = hist.Hist(mc[f"ee{cat}/"+group])
            mm = hist.Hist(mc[f"mm{cat}/"+group])
            mc_stack[cat][group] = ee+mm
    
    for cat in cats:     
        ee = hist.Hist(mc[f"ee{cat}/reducible"])
        mm = hist.Hist(mc[f"mm{cat}/reducible"])
        reducible_stack[cat] = ee + mm
        
    stacks[btag] = {"mc": mc_stack, "reducible": reducible_stack}

In [ ]:
for btag, s in stacks.items():
    mc_stack, reducible_stack = s["mc"], s["reducible"]
    for cat in cats:
        colors = {
            "SM-H(125)": "#E9D8A6",
            "ZZ": "#94D2BD",
            "WZ": "#9b2226",
            "tt": "#0A9396",
            "VVV": "#bb3e03",
            "Reducible": "#074e67"
        }

        stack = mc_stack[cat]
        stack["ZZ"] = stack["ggZZ"] + stack["ZZ"]
        stack["tt"] = stack["TTZ"]
        stack["SM-H(125)"] = stack["ZHtt"] + stack["TTHtt"] + stack["WHWW"] + stack["ZHWW"] + stack["ggZHWW"] + stack["ggHZZ"]
        stack["Reducible"] = reducible_stack[cat]

        stack = {k: v for k, v in stack.items() if k in colors.keys()}
        stack = {k: v for k, v in sorted(stack.items(), key=lambda h: -h[1].sum())}
        colors = {k: colors[k] for k in stack.keys()}
        stack = {c: stack[c] for c in colors.keys() if c in stack.keys()}
        stack = hist.Stack.from_dict(stack)
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            figsize=(11,11),
            dpi=200,
        )

        ax.set_prop_cycle(cycler(color=list(colors.values())))
        stack.plot(ax=ax, stack=True, histtype="fill", edgecolor=(0, 0, 0, 0.3))

        ax.set_ylim([10**-3, 1.5*10**1])
        ax.set_xscale("log")
        ax.set_yscale("log")
        #ax.set_ylim([10e-4, 10e2])
        ax.legend(loc="best", prop={"size": 14}, frameon=True)
        ax.get_legend().set_title(f"ll{cat}, {btag}")
        hep.cms.label("Preliminary", data=True, lumi=lumi, year=year, ax=ax)



### plt.plot(np.arange(0,100,1), np.arange(0,100,1), "#FFE293", linewidth=100)

In [ ]:
for cat in cats:
    colors = {
        "SM-H(125)": "#E9D8A6",
        "ZZ": "#94D2BD",
        "WZ": "#9b2226",
        "tt": "#0A9396",
        "VVV": "#bb3e03",
        "Reducible": "#074e67"
    }
    
    stack = mc_stack[cat]
    stack["ZZ"] = stack["ggZZ"] + stack["ZZ"]
    stack["tt"] = stack["TTZ"]
    #stack["SM-H(125)"] = stack["ZHtt"] + stack["TTHtt"] + stack["WHWW"] + stack["ZHWW"] + stack["ggZHWW"] + stack["ggHZZ"]
    stack["Reducible"] = reducible_stack[cat]
    
    #stack = {k: v for k, v in stack.items() if k in colors.keys()}
    stack = {k: v for k, v in sorted(stack.items(), key=lambda h: h[1].sum())}
    #for k, v in stack.items(): print(k, v.sum())
    #colors = {k: colors[k] for k in stack.keys()}
    #stack = {c: stack[c] for c in colors.keys() if c in stack.keys()}
    stack = hist.Stack.from_dict(stack)
    fig, ax = plt.subplots(
        nrows=1,
        ncols=1,
        figsize=(11,11),
        dpi=200,
    )
    
    #ax.set_prop_cycle(cycler(color=list(colors.values())))
    stack.plot(ax=ax, stack=True, histtype="fill", edgecolor=(0, 0, 0, 0.3))
    
    ax.set_ylim([10**-3, 1.5*10**1])
    ax.set_xscale("log")
    ax.set_yscale("log")
    #ax.set_ylim([10e-4, 10e2])
    ax.legend(loc="best", prop={"size": 14}, frameon=True)
    ax.get_legend().set_title(f"ll{cat}")
    hep.cms.label("Preliminary", data=True, lumi=lumi, year=year, ax=ax)



In [ ]:
for cat in cats:
    colors = {
        "SM-H(125)": "#E9D8A6",
        "ZZ": "#94D2BD",
        "WZ": "#9b2226",
        "tt": "#0A9396",
        "VVV": "#bb3e03",
        "Reducible": "#074e67"
    }
    
    stack = mc_stack[cat]
    stack["ZZ"] = stack["ggZZ"] + stack["ZZ"]
    stack["tt"] = stack["TTZ"]
    #stack["SM-H(125)"] = stack["ZHtt"] + stack["TTHtt"] + stack["WHWW"] + stack["ZHWW"] + stack["ggZHWW"] + stack["ggHZZ"]
    stack["Reducible"] = reducible_stack[cat]
    
    #stack = {k: v for k, v in stack.items() if k in colors.keys()}
    stack = {k: v for k, v in sorted(stack.items(), key=lambda h: h[1].sum())}
    for k, v in stack.items(): print(k, v.sum())
    #colors = {k: colors[k] for k in stack.keys()}
    #stack = {c: stack[c] for c in colors.keys() if c in stack.keys()}
    stack = hist.Stack.from_dict(stack)
    fig, ax = plt.subplots(
        nrows=1,
        ncols=1,
        figsize=(11,11),
        dpi=200,
    )
    
    #ax.set_prop_cycle(cycler(color=list(colors.values())))
    stack.plot(ax=ax, stack=True, histtype="fill", edgecolor=(0, 0, 0, 0.3))
    
    ax.set_ylim([10**-3, 1.5*10**1])
    ax.set_xscale("log")
    ax.set_yscale("log")
    #ax.set_ylim([10e-4, 10e2])
    ax.legend(loc="best", prop={"size": 14}, frameon=True)
    ax.get_legend().set_title(f"ll{cat}")
    hep.cms.label("Preliminary", data=True, lumi=lumi, year=year, ax=ax)
